# Cеточные модели уравнений с частными производными

## Лабораторная №1 (Методы Эйлера и Эйлера Коши)

Выполнил Мусатов Данила Юрьевич КМБО-03-18

# Задание №1

Требуется найти функцию $Y=Y(x), $ удовлетворяющую уравнению $ Y'=f(x,Y) $ и принимающую при $x=x_0$ заданное значение $Y_0 : Y(x_0) = Y_0 $


Вариант 2 : $$ y'(x) =\frac{(-y(x))}{2x}+x^2, \ y (1) = 1, x \in{[1,2]}$$ 

Алгоритм Эйлера. <br>Общая формула для приближения решения будет иметь вид: $ y_{i+1} = y_i + hf(x_i, y_i) $  
Полученная аппроксимация имеет первый порядок  $ - \ $ точность метода  O(h)

## Точное аналитическое решение
<br>$$y{\left(1 \right)} = 1$$<br><br>$$y{\left(x \right)} = \frac{C_{1}}{\sqrt{x}} + \frac{2 x^{3}}{7}$$<br><br>$$1 = \frac{1}{\sqrt{1}} C_{1} + \frac{2 \cdot 1^{3}}{7}$$<br><br>$$C_{1} = \frac{5}{7}$$<br><br>$$y{\left(x \right)} = \frac{2 x^{3}}{7} + \frac{5}{7 \sqrt{x}}$$<br><br><a data-pod="funcgraphs.xy" data-input='{"function": "2*x^3/7 + 5/(7*sqrt(x))", "x": "x"}' style="display: none;" >

# Реализация

In [1]:
#! jupyter nbextension install --user --py widgetsnbextension

In [2]:
#! jupyter nbextension enable --py widgetsnbextension

In [3]:
# модуль для отображение виджета
from IPython.display import display
import ipywidgets as ip_w
import numpy as np
import matplotlib.pyplot as plt
import math

In [4]:
def Method_Euler_without_recalc(f, y0, a, b, h):
    #Euler method withot recalc
    xs = np.arange(a, b + h, h)
    ys = [y0]
    for x in xs[1:]:
        ys.append(ys[-1] + h*f(x, ys[-1]))
    return xs, np.array(ys)  

In [5]:
f = lambda x, y : ((-y)/(2*x)+x**2) #исходная произовдная функции
A = 1 #левый конец интервала
B = 2 #правый конец интервала
Y0 = 1 #начальная точка

In [6]:
def source_func(x): # Аналитическое решение задачи
    return 2*(x**3)/7 + 5/(7*np.sqrt(x))

In [7]:
# Функция для оценки максимального отклонения от аналитического решения
def max_error(xs, ys, source_func):
    return max(np.fabs(ys - source_func(xs)))

In [8]:
def source_without_recalc(y0, a, b, h): #сравнение между методом Эйлера и точным значением
    xs, ys = Method_Euler_without_recalc(f, y0, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Метод эйлера без перерасчета с шагом " + str(h))
    plt.plot(xs, source_func(xs), color="green", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys, source_func))

In [9]:
ip_w.interact(source_without_recalc,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

## Оценка погрешности с помощью правила Рунге

$$error = \frac{| y_h - y_{h/2}|}{2^p - 1}$$ 

In [10]:
def Runge_rule(ys1, ys2, p=1):
    return max(np.fabs(ys1 - ys2))/(2**p - 1)

Для метода Эйлера без перерасчета порядок точности равен 1

In [11]:
def Euler_estimation_error_Ruge_depending_on_h(y0,a,b,h):
    _, ys1 = Method_Euler_without_recalc(f, y0, a, b, h)
    _, ys2 = Method_Euler_without_recalc(f, Y0, A, B, h/2)
    if(len(ys1)!=len(ys2[::2])):
        ys1=np.delete(ys1,len(ys1)-1,0)
    err1 = Runge_rule(ys1, ys2[::2])
    print(f" Ошибка по Правило рунге для шага {h} равна {round(err1, 7)}" )

In [12]:
ip_w.interact(Euler_estimation_error_Ruge_depending_on_h,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), p=ip_w.fixed(1), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Ошибка по Правило рунге для шага 0.01 равна 0.0086439<br>
Ошибка по Правило рунге для шага 0.1 равна 0.088106

## Метод эйлера с пересчётом

Модификация алгоритма Эйлера. формула с пересчетом
Общая рекуррентная формула имеет вид:
$$ y_{i+1} = y_i + \frac{h}{2} (f(x_i,y_i) + f(x_{i+1}, y_i + hf(x_i, y_i))).$$ 

Точность данного метода  $ - \ O(h^2)$

In [13]:
def Method_Euler_with_recalc(f, y0, a, b, h):
    #Modification Euler method
    ys = [y0]
    xs = np.arange(a, b + h, h)
    for x in xs[1: ]:
        ys.append(ys[-1] + h/2 * (f(x, ys[-1]) + f(x + h, ys[-1] + h*f(x, ys[-1]))))
    return xs, np.array(ys)  

In [14]:
def comparison_source_with_recalc(y0, a, b, h):
    xs, ys = Method_Euler_with_recalc(f, y0, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Метод эйлера С перерасчета с шагом " + str(h))
    plt.plot(xs, source_func(xs), color="green", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys, source_func))

In [15]:
ip_w.interact(comparison_source_with_recalc,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

## Оценка погрешности с помощью правила Рунге

$$error = \frac{| y_h - y_{h/2}|}{2^p - 1}$$ 

Для метода Эйлера c перерасчетом порядок точности равен 2

In [16]:
def Euler_2_estimation_error_Ruge_depending_on_h(y0,a,b,h):
    _, ys1 = Method_Euler_with_recalc(f, y0, a, b, h)
    _, ys2 = Method_Euler_with_recalc(f, Y0, A, B, h/2)
    if(len(ys1)!=len(ys2[::2])):
        ys1=np.delete(ys1,len(ys1)-1,0)
    err1 = Runge_rule(ys1, ys2[::2],2)
    print(f" Ошибка по Правилу рунге для шага {h} равна {round(err1, 7)}" )

In [17]:
ip_w.interact(Euler_2_estimation_error_Ruge_depending_on_h,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), p=ip_w.fixed(1), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Ошибка по Правило рунге для шага 0.01 равна 0.004944<br>
Ошибка по Правило рунге для шага 0.1 равна 0.05211

## Нахождение решения с помощью библиотеки scipy

In [18]:
from scipy.integrate import odeint as ODE

In [19]:
g = lambda y, x : f(x,y)
def scipy_ODE(f, y0, a, b, h):
    xs = np.arange(a, b+h, h)
    ys = ODE(f, y0, xs)
    return xs, ys

In [20]:
def comparison_source_with_Scipy(y0, a, b, h):
    xs, ys = scipy_ODE(g, Y0, A, B, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Решение с помощью модуля Scipy " + str(h))
    plt.plot(xs, source_func(xs), color="green", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()    
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys.flat, source_func))


In [21]:
ip_w.interact(comparison_source_with_Scipy,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Значения от аналитического решения отличаются на 10^(-8)